* Loads the xlsx file into a df

In [1]:
import openpyxl
import pandas as pd

wb = openpyxl.load_workbook("MSR2021.xlsx")

sheet = wb.active  


# getting the non_header values 
data_no_headers = [row for i,row in enumerate(sheet.iter_rows(values_only=True)) if i>0]
#getting the headers
headers = [row for i, row in enumerate(sheet.iter_rows(values_only=True)) if i==0]
headers= [[head.lower() for head in header] for header in headers]

df = pd.DataFrame(data_no_headers, columns= headers)
df.head()

,restaurant,link,michelin guide,address,price,cuisine,michelin guide point of view,facilities & services,contact number,website,opening hours,michelin stars,location_lat,location_long
0,Pelagos,https://guide.michelin.com/en/attica/athens/re...,MICHELIN GUIDE GREECE,"Four Seasons Astir Palace Hotel, 40 Apollonos ...",80 - 155 EUR,Seafood,‘Ocean’ is a fitting name for this hotel resta...,\nT\nAir conditioning\nV\nAmerican Express cre...,+30 21 0890 1190,www.fourseasons.com/athens/dining/restaurants/...,"Closed: 1-31 January, Monday, lunch: Tuesday-S...",1,37.983941,23.728305
1,Botrini's,https://guide.michelin.com/en/attica/athens/re...,MICHELIN GUIDE GREECE,"24b Vasileos Georgiou, Athens, 152 33, Greece",90 - 120 EUR,Mediterranean Cuisine,Ask for a spot in the romantic garden of this ...,\nT\nAir conditioning\nCredit card / Debit car...,+30 21 0685 7323,https://www.botrinis.com/,"Closed: 17-25 April, Monday, lunch: Tuesday-Sa...",1,37.983941,23.728305
2,Hytra,https://guide.michelin.com/en/attica/athens/re...,MICHELIN GUIDE GREECE,"107-109 Syngrou Avenue, Athens, 11 745, Greece",55 - 90 EUR,Modern Cuisine,"Once the sun goes down, they fold back the roo...",\nT\nAir conditioning\nV\nAmerican Express cre...,+30 21 0331 6767,https://www.hytra.gr/,Closed: lunch: Monday-Sunday,1,37.994860,23.755160
3,CTC,https://guide.michelin.com/en/attica/athens/re...,MICHELIN GUIDE GREECE,"15 Plateon, Athens, 10435, Greece",72 EUR,Modern Cuisine,CTC’s move to a new location means that it now...,\nT\nAir conditioning\nV\nAmerican Express cre...,+30 21 0722 8812,https://www.ctc-restaurant.com/,"Closed: Monday, lunch: Tuesday-Saturday, Sunday",1,37.981129,23.716432
4,Varoulko Seaside,https://guide.michelin.com/en/attica/athens/re...,MICHELIN GUIDE GREECE,"Akti Koumoundourou, 54-56 Mikrolimano Marina, ...",43 - 60 EUR,Seafood,"This lively, bustling restaurant sits in a gre...",\nT\nAir conditioning\nV\nAmerican Express cre...,+30 21 0522 8400,https://www.varoulko.gr/,Closed: lunch: Monday-Saturday,1,37.983941,23.728305


* converts the currencies into USD

In [2]:
#function to convert all the prices to USD
def convert_to_USD(currency_li):
    new_li=[]
    #iterate over the inputted list of currency and price ranges
    for row in currency_li:
        #get the currency
        currency = str(row.split()[-1]).strip()
        #get the lower price value
        lower_cost = int(row.split()[0].replace(",",""))
        #try to get the higher price val
        try:
            higher_cost =  int(row.split()[2])
        #if there isn't a higher price val, make the higher price val the lower price val: range = 0
        except:
            higher_cost= lower_cost
        #make a dict of currencies as the key and exchange rates to USD as vals
        exchange_rates = {'EUR': 1.05,'HRK': .14244, 'HUF': .0026, 'PLN': .25, 'CZK': .042, 'CHF': 1.11, 'RUB': .011, 'DKK': .14, 'SEK': .094, 'USD': 1 , 'JPY': .0066, 'TWD': .031, 'SGD': .75, 'KRW': .00069, 'CNY': .14, 'HKD': .13, 'MOP': .12472, 'ISK': .0071, 'GBP': 1.26, 'NOK': .09, 'THB': .03, 'BRL': .17548}
        #index the dict using the currency to find the base to multiply with
        base = exchange_rates[currency]
        #multiply both ints by the base
        lower_cost *= base
        higher_cost *= base
        #new list to append vals to
        li = []
        li.append(round(lower_cost,2))
        li.append(round(higher_cost,2))
        #append the list to the new list
        new_li.append(li)
    #return the nested list
    return new_li      

* only takes the first cuisine

In [3]:
li=[]
for i in df['cuisine'].values:
    li.append(i[0].split(',')[0])
    
    
df['cuisine']= li

* renames the cols

In [4]:
df['stars']= df['michelin stars']
df['long'] = df['location_long']
df['lat']=df['location_lat']


* makes lower price and higher price cols
* renames them
* selects the cols I want and makes it into a new df called mg

In [5]:
df[['Lower_Price_USD', 'Higher_Price_USD']] = pd.DataFrame(convert_to_USD(df.iloc[:, 4].astype(str)))
df['lower_price']= df["Lower_Price_USD"]
df['higher_price']= df["Higher_Price_USD"]

columns = ["restaurant", "cuisine", "stars", "long", "lat", "lower_price", "higher_price"]

# Reindex the DataFrame to include only the selected columns
mg = df[columns]

In [6]:
mg

,restaurant,cuisine,stars,long,lat,lower_price,higher_price
0,Pelagos,Seafood,1,23.728305,37.983941,84.00,162.75
1,Botrini's,Mediterranean Cuisine,1,23.728305,37.983941,94.50,126.00
2,Hytra,Modern Cuisine,1,23.755160,37.994860,57.75,94.50
3,CTC,Modern Cuisine,1,23.716432,37.981129,75.60,75.60
4,Varoulko Seaside,Seafood,1,23.728305,37.983941,45.15,63.00
...,...,...,...,...,...,...,...
3150,The French Laundry,Contemporary,3,-122.365000,38.404393,350.00,450.00
3151,Quince,Contemporary,3,-122.403269,37.797432,195.00,298.00
3152,Atelier Crenn,Contemporary,3,-122.435844,37.798322,298.00,298.00
3153,Benu,Asian,3,-122.399080,37.785462,350.00,350.00


In [7]:
import os

file_path = "Michelin_Guide_2021.csv"
file_size = os.path.getsize(file_path)  # Size in bytes

print(f"File size: {file_size} bytes")

File size: 199440 bytes


In [8]:
df.head()

,restaurant,link,michelin guide,address,price,cuisine,michelin guide point of view,facilities & services,contact number,website,...,michelin stars,location_lat,location_long,stars,long,lat,Lower_Price_USD,Higher_Price_USD,lower_price,higher_price
0,Pelagos,https://guide.michelin.com/en/attica/athens/re...,MICHELIN GUIDE GREECE,"Four Seasons Astir Palace Hotel, 40 Apollonos ...",80 - 155 EUR,Seafood,‘Ocean’ is a fitting name for this hotel resta...,\nT\nAir conditioning\nV\nAmerican Express cre...,+30 21 0890 1190,www.fourseasons.com/athens/dining/restaurants/...,...,1,37.983941,23.728305,1,23.728305,37.983941,84.00,162.75,84.00,162.75
1,Botrini's,https://guide.michelin.com/en/attica/athens/re...,MICHELIN GUIDE GREECE,"24b Vasileos Georgiou, Athens, 152 33, Greece",90 - 120 EUR,Mediterranean Cuisine,Ask for a spot in the romantic garden of this ...,\nT\nAir conditioning\nCredit card / Debit car...,+30 21 0685 7323,https://www.botrinis.com/,...,1,37.983941,23.728305,1,23.728305,37.983941,94.50,126.00,94.50,126.00
2,Hytra,https://guide.michelin.com/en/attica/athens/re...,MICHELIN GUIDE GREECE,"107-109 Syngrou Avenue, Athens, 11 745, Greece",55 - 90 EUR,Modern Cuisine,"Once the sun goes down, they fold back the roo...",\nT\nAir conditioning\nV\nAmerican Express cre...,+30 21 0331 6767,https://www.hytra.gr/,...,1,37.994860,23.755160,1,23.755160,37.994860,57.75,94.50,57.75,94.50
3,CTC,https://guide.michelin.com/en/attica/athens/re...,MICHELIN GUIDE GREECE,"15 Plateon, Athens, 10435, Greece",72 EUR,Modern Cuisine,CTC’s move to a new location means that it now...,\nT\nAir conditioning\nV\nAmerican Express cre...,+30 21 0722 8812,https://www.ctc-restaurant.com/,...,1,37.981129,23.716432,1,23.716432,37.981129,75.60,75.60,75.60,75.60
4,Varoulko Seaside,https://guide.michelin.com/en/attica/athens/re...,MICHELIN GUIDE GREECE,"Akti Koumoundourou, 54-56 Mikrolimano Marina, ...",43 - 60 EUR,Seafood,"This lively, bustling restaurant sits in a gre...",\nT\nAir conditioning\nV\nAmerican Express cre...,+30 21 0522 8400,https://www.varoulko.gr/,...,1,37.983941,23.728305,1,23.728305,37.983941,45.15,63.00,45.15,63.00


In [9]:
mg.to_csv("Michelin_Guide_2021.csv", index=False)